In [24]:
import numpy as np
import os
import time
folder = "Problems"

time_limit = 300
k = 3

In [19]:
def getpermutation(data):
    cur_ans = []
    if data == []:
        return [[]]
    for i in range(len(data)):
        last_answer = getpermutation(data[:i]+data[i+1:])
        
        for j in range(len(last_answer)): 
            last_answer[j].append(data[i])

        cur_ans.extend(last_answer)
    return cur_ans

In [20]:
def getpermutation2(start, end, k):
    if k == 0:
        return [[]]
    ans = []
    for i in range(start,end-(k-1)):
        last_ans = getpermutation2(i+2, end,k-1)
        for j in last_ans:
            ans.append([i] + j)
    return ans

In [25]:
result = [[] for _ in os.listdir(folder)]
for step, dirs in enumerate(sorted(os.listdir(folder), key = lambda x: int(x))):
    print("Test with number of {} begin!".format(dirs))
    cur_dir = os.path.join(folder, dirs)
    for files in os.listdir(cur_dir):
        start_time = time.time()
        path = os.path.join(cur_dir, files)
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read().split()
            data = list(map(float, content))
            number = int(data[0])
            distances = np.zeros((number,number))
        for i in range(number):
            for j in range(number):
                distances[i][j] = data[i*number+j+1]
        
        ans = np.inf
        ans_route = [""]
        choices = getpermutation2(0, number, k)
        for _ in range(10000):
            if time.time() - start_time > time_limit:
                break
            order = list(np.arange(1, number))
            np.random.shuffle(order)
            route = [0] + order + [0]
            while True:
                if time.time() - start_time > time_limit:
                    break
                improvement = 0
                
                for edges in choices:
                    if time.time() - start_time > time_limit:
                        break
                    current = 0
                    data = []
                    for i in range(len(edges)):
                        current += distances[route[edges[i]]][route[edges[i]+1]] 
                        if i != len(edges) -1:
                            data.append([edges[i]+1, edges[i+1]])
                    # print(data)
                    # data = [[i+1, j], [j+1, k]]
                    permutation = getpermutation(data)
                    for choice in permutation:
                        
                        for chioce2 in range(0, 2**len(data)):
                            roads = []
                            for point in range(len(data)):
                                roads.append(data[point][chioce2>>point&1])
                                roads.append(data[point][abs((chioce2>>point&1)-1)])
                            next = distances[route[edges[0]]][route[roads[0]]] + distances[route[roads[-1]]][route[edges[-1]+1]]

                            for s in range(1,len(roads)-1,2):
                                next += distances[route[roads[s]]][route[roads[s+1]]]
                            # next = distances[route[i]][route[roads[0]]] + distances[route[roads[1]]][route[roads[2]]] + distances[route[roads[3]]][route[k+1]]
                            if current - next > improvement:
                                next_step = [edges[0]] + roads + [edges[-1]+1]
                                improvement = current - next
                
                if improvement < 10**(-6):
                    break
                new_route = []
                for i in range(1, len(next_step)-1, 2):
                    if next_step[i+1]>next_step[i]:
                        new_route.extend(route[next_step[i]:next_step[i+1]+1])
                    else:
                        new_route.extend(route[next_step[i]:next_step[i+1]-1:-1])


                odd_route = route[:]

                route[next_step[0]+1 :next_step[-1]] = new_route

            cur = sum([distances[route[i]][route[i+1]] for i in range(number)])
            if cur < ans:
                ans = cur   
                ans_route = route 
        print("ans is: {}, route is: {}".format(np.round(ans, 2), '->'.join(map(str, ans_route))))
        result[step].append(ans)
    print()
    print()


Test with number of 10 begin!
ans is: 469.78, route is: 0->6->5->4->2->8->3->9->1->7->0
ans is: 449.22, route is: 0->5->4->2->7->9->3->8->6->1->0
ans is: 416.75, route is: 0->3->9->1->7->5->8->6->4->2->0
ans is: 506.28, route is: 0->2->6->4->5->7->1->3->8->9->0
ans is: 518.05, route is: 0->9->2->8->5->1->3->4->7->6->0
ans is: 420.27, route is: 0->1->2->8->4->9->5->6->7->3->0
ans is: 547.28, route is: 0->7->6->1->4->2->8->3->5->9->0
ans is: 484.33, route is: 0->7->5->3->4->1->8->9->6->2->0
ans is: 735.16, route is: 0->1->2->3->7->4->5->8->9->6->0
ans is: 470.02, route is: 0->8->1->6->4->5->3->9->7->2->0


Test with number of 20 begin!
ans is: 734.74, route is: 0->12->7->11->3->2->19->5->14->8->18->9->16->6->13->1->15->17->10->4->0
ans is: 638.5, route is: 0->11->12->16->5->14->18->2->7->10->17->13->3->9->15->4->6->19->1->8->0
ans is: 733.38, route is: 0->7->12->18->11->13->3->19->14->6->17->1->15->2->5->16->10->8->4->9->0
ans is: 455.43, route is: 0->16->11->9->8->12->18->1->6->13->10->

KeyboardInterrupt: 

In [23]:
from openpyxl import load_workbook

# 载入已存在的xlsx文件
wb = load_workbook('result.xlsx')

# 选择一个工作表
ws = wb["300s"]

# 写入数据到第A列（你可以更改列名来选择不同的列）
ws[f'H{1}'] = "3_opt"
for i, item in enumerate(result, start=2):  # start=1 表示从第一行开始
    ws[f'H{i}'] = np.mean(item)
# 保存文件
wb.save('result.xlsx')